# IBM Applied Data Science Capstone
### Week 5 Final notebook
**Opening a New Shopping Mall in Denver, Colorado**
- Build a dataframe of neighborhoods in Denver, Colorado by web scraping the data from Wikipedia page
- Get the geographical coordinates of the neighborhoods
- Obtain the venue data for the neighborhoods from Foursquare API
- Explore and cluster the neighborhoods
- Select the best cluster to open a new shopping mall
***
### 1. Import libraries

In [44]:
# Install necessary libraries
!pip install geopy
!pip install geocoder
!pip install folium

You should consider upgrading via the 'c:\users\derek\anaconda3\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\derek\anaconda3\python.exe -m pip install --upgrade pip' command.
You should consider upgrading via the 'c:\users\derek\anaconda3\python.exe -m pip install --upgrade pip' command.


In [45]:
# library to handle data in a vectorized manner
import numpy as np 

# data analysis
import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)

# handle JSON files
import json 

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim 
import geocoder

# handle requests
import requests

# parse HTML and XML documents
from bs4 import BeautifulSoup 

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize 

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering

print("Libraries imported.")

Libraries imported.


### 2. Scrap data from Wikipedia page into a DataFrame

In [46]:
# send the GET request
data = requests.get("https://en.wikipedia.org/wiki/Category:Neighborhoods_in_Denver").text

In [47]:
# parse data from the html into a beautifulsoup object
soup = BeautifulSoup(data, 'html.parser')

In [48]:
# create a list to store neighborhood data
neighborhoodList = []

In [49]:
# append the data into the list
for row in soup.find_all("div", class_="mw-category")[0].findAll("li"):
    neighborhoodList.append(row.text)

In [50]:
# create a new DataFrame from the list
denver_df = pd.DataFrame({"Neighborhood": neighborhoodList})

denver_df.head()

,Neighborhood
0,List of neighborhoods in Denver
1,"Alamo Placita, Denver"
2,"Athmar Park, Denver"
3,"Auraria, Denver"
4,"Baker, Denver"


In [51]:
# print the number of rows of the dataframe
denver_df.shape

(52, 1)

### 3. Get the geographical coordinates

In [52]:
# define a function to get coordinates
def get_latlng(neighborhood):
    # initialize your variable to None
    lat_lng_coords = None
    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.arcgis('{}, Denver, Colorado'.format(neighborhood))
        lat_lng_coords = g.latlng
    return lat_lng_coords

In [53]:
# call the function to get the coordinates, store in a new list using list comprehension
coords = [ get_latlng(neighborhood) for neighborhood in denver_df["Neighborhood"].tolist() ]

In [54]:
coords

[[39.74308700000006, -104.97893599999998],
 [39.721070000000054, -104.97521999999998],
 [39.70396000000005, -105.01038999999997],
 [39.745750000000044, -105.00996999999995],
 [39.71117000000004, -104.99208999999996],
 [39.71816000000007, -105.03261999999995],
 [39.70384000000007, -104.94973999999996],
 [39.77666000000005, -105.03956999999997],
 [39.733700000000056, -104.97956999999997],
 [39.76013000000006, -104.87849999999997],
 [39.78741000000008, -105.01755999999995],
 [39.72911000000005, -104.96816999999999],
 [39.717820000000074, -104.94872999999995],
 [39.74001000000004, -104.99201999999997],
 [39.74545000000006, -104.96719999999999],
 [39.74379000000005, -104.94824999999997],
 [39.73526000000004, -104.99057999999997],
 [39.76693000000006, -104.95050999999995],
 [39.76633000000004, -104.96593999999999],
 [39.73359000000005, -104.95000999999996],
 [39.721960000000024, -104.96551999999997],
 [39.74822000000006, -104.99412999999998],
 [39.781960000000026, -104.95909999999998],
 [39.

In [55]:
# create temporary dataframe to populate the coordinates into Latitude and Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])

In [56]:
# merge the coordinates into the original dataframe
denver_df['Latitude'] = df_coords['Latitude']
denver_df['Longitude'] = df_coords['Longitude']

In [57]:
# check the neighborhoods and the coordinates
print(denver_df.shape)
denver_df

(52, 3)


,Neighborhood,Latitude,Longitude
0,List of neighborhoods in Denver,39.743087,-104.978936
1,"Alamo Placita, Denver",39.721070,-104.975220
2,"Athmar Park, Denver",39.703960,-105.010390
3,"Auraria, Denver",39.745750,-105.009970
4,"Baker, Denver",39.711170,-104.992090
5,"Barnum, Denver",39.718160,-105.032620
6,"Belcaro, Denver",39.703840,-104.949740
7,"Berkeley, Denver",39.776660,-105.039570
8,"Capitol Hill, Denver",39.733700,-104.979570
9,"Central Park, Denver",39.760130,-104.878500


In [58]:
# Remove First row
denver_df = denver_df.iloc[1:]
denver_df.head()

,Neighborhood,Latitude,Longitude
1,"Alamo Placita, Denver",39.72107,-104.97522
2,"Athmar Park, Denver",39.70396,-105.01039
3,"Auraria, Denver",39.74575,-105.00997
4,"Baker, Denver",39.71117,-104.99209
5,"Barnum, Denver",39.71816,-105.03262


In [59]:
# save the DataFrame as CSV file
denver_df.to_csv("denver_df.csv", index=False)

### 4. Create a map of Kuala Lumpur with neighborhoods superimposed on top

In [60]:
# get the coordinates of Kuala Lumpur
address = 'Denver'

geolocator = Nominatim(user_agent= "http")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Denver, Colorado {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Denver, Colorado 39.7392364, -104.9848623.


In [61]:
# create map of Toronto using latitude and longitude values
denver_map = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, neighborhood in zip(denver_df['Latitude'], denver_df['Longitude'], denver_df['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_kl)  
    
denver_map

In [62]:
# save the map as HTML file
denver_map.save('denver_map.html')

### 5. Use the Foursquare API to explore the neighborhoods

In [85]:
# define Foursquare Credentials and Version
CLIENT_ID = 'your Foursquare ID' # your Foursquare ID
CLIENT_SECRET = 'your Foursquare Secret' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: your Foursquare ID
CLIENT_SECRET:your Foursquare Secret


**Now, let's get the top 100 venues that are within a radius of 2000 meters.**

In [64]:
radius = 1000
LIMIT = 100

venues = []

for lat, long, neighborhood in zip(denver_df['Latitude'], denver_df['Longitude'], denver_df['Neighborhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
    
    # make the GET request
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighborhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [65]:
# convert the venues list into a new DataFrame
venues_df = pd.DataFrame(venues)

# define the column names
venues_df.columns = ['Neighborhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(venues_df.shape)
venues_df.head()

(2845, 7)


,Neighborhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,"Alamo Placita, Denver",39.72107,-104.97522,Alamo Placita Park,39.721457,-104.975283,Park
1,"Alamo Placita, Denver",39.72107,-104.97522,Cafe Byblos Greek & Lebanese,39.722045,-104.973853,Mediterranean Restaurant
2,"Alamo Placita, Denver",39.72107,-104.97522,Table 6,39.725824,-104.974280,American Restaurant
3,"Alamo Placita, Denver",39.72107,-104.97522,Angelo's Taverna,39.725551,-104.979204,Italian Restaurant
4,"Alamo Placita, Denver",39.72107,-104.97522,Firefly Massage,39.719966,-104.976688,Massage Studio


**Let's check how many venues were returned for each neighorhood**

In [66]:
venues_df.groupby(["Neighborhood"]).count()

,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
Neighborhood,,,,,,
"Alamo Placita, Denver",50,50,50,50,50,50
"Athmar Park, Denver",31,31,31,31,31,31
"Auraria, Denver",72,72,72,72,72,72
"Baker, Denver",84,84,84,84,84,84
"Barnum, Denver",28,28,28,28,28,28
"Belcaro, Denver",46,46,46,46,46,46
"Berkeley, Denver",85,85,85,85,85,85
"Capitol Hill, Denver",100,100,100,100,100,100
"Central Park, Denver",28,28,28,28,28,28


**Let's find out how many unique categories can be curated from all the returned venues**

In [67]:
print('There are {} uniques categories.'.format(len(venues_df['VenueCategory'].unique())))

There are 291 uniques categories.


In [68]:
# print out the list of categories
venues_df['VenueCategory'].unique()[:50]

array(['Park', 'Mediterranean Restaurant', 'American Restaurant',
       'Italian Restaurant', 'Massage Studio', 'Breakfast Spot',
       'Wine Shop', 'Smoke Shop', 'Café', 'New American Restaurant',
       'Mexican Restaurant', 'Pizza Place', 'Bagel Shop', 'Grocery Store',
       'Coffee Shop', 'Indian Restaurant', 'Sandwich Place', 'Bar',
       'Liquor Store', 'Ramen Restaurant', 'Frozen Yogurt Shop',
       'Video Store', 'Japanese Restaurant', 'Sushi Restaurant',
       'Noodle House', 'Fishing Spot', 'BBQ Joint', 'Karaoke Bar', 'Bank',
       'Cosmetics Shop', 'Fish Market', 'Fast Food Restaurant',
       'Bus Stop', 'Vietnamese Restaurant', 'Dim Sum Restaurant',
       'Warehouse Store', 'Brewery', 'Video Game Store', 'Supermarket',
       'Asian Restaurant', 'Financial or Legal Service', 'Wings Joint',
       'Discount Store', 'Convenience Store', 'Bakery', 'Event Service',
       'Deli / Bodega', 'Flower Shop', 'Herbs & Spices Store',
       'Construction & Landscaping'], dtyp

In [69]:
# check if the results contain "Shopping Mall"
"Neighborhood" in venues_df['VenueCategory'].unique()

True

### 6. Analyze Each Neighborhood

In [70]:
# one hot encoding
denver_onehot = pd.get_dummies(venues_df[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
denver_onehot['Neighborhoods'] = venues_df['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [denver_onehot.columns[-1]] + list(denver_onehot.columns[:-1])
denver_onehot = denver_onehot[fixed_columns]

print(denver_onehot.shape)
denver_onehot.head()

(2845, 292)


,Neighborhoods,ATM,Accessories Store,Adult Education Center,Airport,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Alternative Healer,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Shop,Ski Trail,Smoke Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Trade School,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Rest

**Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category**

In [71]:
denver_grouped = denver_onehot.groupby(["Neighborhoods"]).mean().reset_index()

print(denver_grouped.shape)
denver_grouped

(51, 292)


,Neighborhoods,ATM,Accessories Store,Adult Education Center,Airport,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,Alternative Healer,American Restaurant,Aquarium,Arcade,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Bookstore,Botanical Garden,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Caribbean Restaurant,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Fabric Shop,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Truck,Football Stadium,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hockey Arena,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Marijuana Dispensary,Market,Martial Arts School,Massage Studio,Mattress Store,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Mongolian Restaurant,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paper / Office Supplies Store,Park,Performing Arts Venue,Pet Café,Pet Store,Pharmacy,Pizza Place,Planetarium,Playground,Plaza,Poke Place,Pool,Print Shop,Pub,Ramen Restaurant,Record Shop,Recreation Center,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Ski Shop,Ski Trail,Smoke Shop,Snack Place,Soup Place,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Storage Facility,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tailor Shop,Tanning Salon,Tapas Restaurant,Tea Room,Tennis Court,Tex-Mex Restaurant,Thai Restaurant,Theater,Theme Park,Theme Park Ride / Attraction,Thrift / Vintage Store,Toy / Game Store,Track,Trade School,Trail,Train Station,Vegetarian / Vegan Restaurant,Veterinarian,Video Game Store,Video Store,Vietnamese Rest

In [72]:
len(denver_grouped[denver_grouped["Shopping Mall"] > 0])

4

**Create a new DataFrame for Shopping Mall data only**

In [73]:
denver_mall = denver_grouped[["Neighborhoods","Shopping Mall"]]

In [74]:
denver_mall.head()

,Neighborhoods,Shopping Mall
0,"Alamo Placita, Denver",0.0
1,"Athmar Park, Denver",0.0
2,"Auraria, Denver",0.0
3,"Baker, Denver",0.0
4,"Barnum, Denver",0.0


### 7. Cluster Neighborhoods
Run k-means to cluster the neighborhoods in Kuala Lumpur into 3 clusters.

In [75]:
# set number of clusters
kclusters = 3

denver_clustering = denver_mall.drop(["Neighborhoods"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(denver_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [76]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
denver_merged = denver_mall.copy()

# add clustering labels
denver_merged["Cluster Labels"] = kmeans.labels_

In [77]:
denver_merged.rename(columns={"Neighborhoods": "Neighborhood"}, inplace=True)
denver_merged.head()

,Neighborhood,Shopping Mall,Cluster Labels
0,"Alamo Placita, Denver",0.0,0
1,"Athmar Park, Denver",0.0,0
2,"Auraria, Denver",0.0,0
3,"Baker, Denver",0.0,0
4,"Barnum, Denver",0.0,0


In [78]:
# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
denver_merged = denver_merged.join(denver_df.set_index("Neighborhood"), on="Neighborhood")

print(denver_merged.shape)
denver_merged.head() # check the last columns!

(51, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"Alamo Placita, Denver",0.0,0,39.72107,-104.97522
1,"Athmar Park, Denver",0.0,0,39.70396,-105.01039
2,"Auraria, Denver",0.0,0,39.74575,-105.00997
3,"Baker, Denver",0.0,0,39.71117,-104.99209
4,"Barnum, Denver",0.0,0,39.71816,-105.03262


In [79]:
# sort the results by Cluster Labels
print(denver_merged.shape)
denver_merged.sort_values(["Cluster Labels"], inplace=True)
denver_merged

(51, 5)


,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"Alamo Placita, Denver",0.000000,0,39.721070,-104.975220
27,"Jefferson Park, Denver",0.000000,0,39.751210,-105.021350
28,"Lincoln Park, Denver",0.000000,0,39.728960,-105.002220
29,"LoDo, Denver",0.000000,0,39.752610,-104.999510
30,"Lowry, Denver",0.000000,0,39.716238,-104.908083
31,"Montbello, Denver",0.000000,0,39.793210,-104.833860
32,"Montclair, Denver",0.000000,0,39.731660,-104.913370
34,"Park Hill, Denver",0.000000,0,39.743784,-104.973379
35,"Regis, Denver",0.000000,0,39.787420,-105.040990
36,"Ruby Hill, Denver",0.000000,0,39.691060,-105.008740


**Finally, let's visualize the resulting clusters**

In [80]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(denver_merged['Latitude'], denver_merged['Longitude'], denver_merged['Neighborhood'], denver_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [81]:
# save the map as HTML file
map_clusters.save('map_clusters.html')

### 8. Examine Clusters

#### Cluster 0

In [82]:
denver_merged.loc[denver_merged['Cluster Labels'] == 0]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
0,"Alamo Placita, Denver",0.0,0,39.721070,-104.975220
27,"Jefferson Park, Denver",0.0,0,39.751210,-105.021350
28,"Lincoln Park, Denver",0.0,0,39.728960,-105.002220
29,"LoDo, Denver",0.0,0,39.752610,-104.999510
30,"Lowry, Denver",0.0,0,39.716238,-104.908083
31,"Montbello, Denver",0.0,0,39.793210,-104.833860
32,"Montclair, Denver",0.0,0,39.731660,-104.913370
34,"Park Hill, Denver",0.0,0,39.743784,-104.973379
35,"Regis, Denver",0.0,0,39.787420,-105.040990
36,"Ruby Hill, Denver",0.0,0,39.691060,-105.008740


#### Cluster 1

In [83]:
denver_merged.loc[denver_merged['Cluster Labels'] == 1]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
15,"Civic Center, Denver (neighborhood)",0.01,1,39.73526,-104.99058
33,"North Capitol Hill, Denver",0.01,1,39.74425,-104.98108
12,"Chinatown, Denver",0.01,1,39.74001,-104.99202


#### Cluster 2

In [84]:
denver_merged.loc[denver_merged['Cluster Labels'] == 2]

,Neighborhood,Shopping Mall,Cluster Labels,Latitude,Longitude
11,"Cherry Creek, Denver",0.010309,2,39.71782,-104.94873


#### Summary:

After reviewing the map of shopping centers in Denver, the majority of the centers are clustered around central denver represented by Cluster 1. Cluster 2 also represents a high concentration of shopping centers. Therefore, Neighborhoods in Cluster 1 and 2 are not viable for new shopping centers.

There are many opportunities for shopping centers in Cluster 0 neighborhoods. Cluster 0 has very few to no shopping centers. The suburbs in Cluster 0 may not be ideal for high traffic shopping centers. However, there are neighborhoods, such as those in Northern Denver near I-70 that are high traffic areas that are not in residential areas. These areas have very little competition and present the most opportunity for building a new shopping center.